Use PowerShell kernel to compile Grains class library project:

In [ ]:
#!pwsh
dotnet build ../src/UrlShortener.Backend.Grains/UrlShortener.Backend.Grains.csproj --nologo --no-self-contained --verbosity minimal

  Determining projects to restore...
  All projects are up-to-date for restore.
  UrlShortener.Backend.Interfaces -> /home/vmusr/Documents/repo/OrleansUrlShortener/src/UrlShortener.Backend.Interfaces/bin/Debug/net6.0/UrlShortener.Backend.Interfaces.dll
  UrlShortener.Backend.Grains -> /home/vmusr/Documents/repo/OrleansUrlShortener/src/UrlShortener.Backend.Grains/bin/Debug/net6.0/UrlShortener.Backend.Grains.dll

Build succeeded.
    0 Warning(s)
    0 Error(s)

Time Elapsed 00:00:03.23


Load built Assembly file:

In [ ]:
#r "../src/UrlShortener.Backend.Grains/bin/Debug/net6.0/UrlShortener.Backend.Grains.dll"

Install Nuget Packages, since .NET Interactive Notebook is not a pure C# project, [we **cannot** use the Build time code generation packages](https://docs.microsoft.com/en-us/dotnet/orleans/grains/code-generation#what-happens-during-build) but instead install `Microsoft.Orleans.OrleansCodeGenerator` that do code generation when both client & silo app starting:

In [ ]:
#r "nuget:Microsoft.Extensions.Hosting"
#r "nuget:Microsoft.Orleans.Server"
#r "nuget:Microsoft.Orleans.OrleansCodeGenerator"

//logging libraries
#r "nuget: Microsoft.Extensions.Logging.Abstractions"
#r "nuget: Microsoft.Extensions.Logging"
#r "nuget: Serilog.Extensions.Logging"
#r "nuget: Serilog.Sinks.Console"

Installed Packages Microsoft.Extensions.Hosting, 6.0.1 Microsoft.Extensions.Logging, 6.0.0 Microsoft.Extensions.Logging.Abstractions, 6.0.1 Microsoft.Orleans.OrleansCodeGenerator, 3.6.3 Microsoft.Orleans.Server, 3.6.3 Serilog.Extensions.Logging, 3.1.0 Serilog.Sinks.Console, 4.0.1

Using some namespace to reduce clutter code and enable hosting extension methods:

In [ ]:
// Orleans Silo
using Microsoft.Extensions.Hosting;
using Orleans;
using Orleans.Hosting;
using Orleans.Runtime;
using Orleans.Configuration;

using Microsoft.Extensions.Logging;
using Serilog;

//Our Orleans Grain & Interface
using UrlShortener.Backend.Grains;
using UrlShortener.Backend.Interfaces;

## Server (Silo Host)
Create [HostBuilder](https://docs.microsoft.com/en-us/dotnet/api/microsoft.extensions.hosting.hostbuilder) and properly configure it:

In [ ]:
Log.Logger = new LoggerConfiguration().WriteTo.Console().CreateLogger();
var codeGenLoggerFactory = 
    LoggerFactory.Create(logBuilder =>{
        logBuilder.AddSerilog();
    });

var hostBuilder = new HostBuilder().UseOrleans(siloBuilder =>{
    siloBuilder.UseLocalhostClustering()
        .Configure<ClusterOptions>(options =>
        {
            options.ClusterId = "local-cluster-01";
            options.ServiceId = "JupyterNotebook_Orleans_Silo";
        })
        .ConfigureApplicationParts(parts =>
        {
            parts.AddApplicationPart(typeof(UrlStoreGrain).Assembly).WithCodeGeneration(codeGenLoggerFactory);
            parts.AddApplicationPart(typeof(IUrlStoreGrain).Assembly).WithCodeGeneration(codeGenLoggerFactory);
        })
        .AddMemoryGrainStorage("url-store");                           
});

Create via host builder and run Silo host, the Grains Server:

In [ ]:
var host = hostBuilder.Build();
host.Start();

[13:29:27 INF] Generating code for assemblies: 
[13:29:28 INF] Runtime code generation for assemblies  UrlShortener.Backend.Grains, Version=1.0.0.0, Culture=neutral, PublicKeyToken=null took 807 milliseconds
[13:29:28 INF] Generating code for assemblies: 
[13:29:28 INF] Runtime code generation for assemblies  UrlShortener.Backend.Interfaces, Version=1.0.0.0, Culture=neutral, PublicKeyToken=null took 455 milliseconds


## Client 
Create [ClientBuilder](https://docs.microsoft.com/en-us/dotnet/api/orleans.clientbuilder) and configure it:

In [ ]:
var clientBuilder = new ClientBuilder().UseLocalhostClustering().Configure<ClusterOptions>(options =>
        {
            options.ClusterId = "dev";
            options.ServiceId = "JupyterNotebook_Orleans_Silo";
        })
        .ConfigureApplicationParts(parts =>
        {
            parts.AddApplicationPart(typeof(IUrlStoreGrain).Assembly).WithCodeGeneration(codeGenLoggerFactory);  // we only needs grin interface code generation.
        });

[13:29:36 INF] Generating code for assemblies: 
[13:29:36 INF] Runtime code generation for assemblies  UrlShortener.Backend.Interfaces, Version=1.0.0.0, Culture=neutral, PublicKeyToken=null took 130 milliseconds


Creat Orleans client and get the *RPC Client Stub* object (Grain method's invocation instance) via a defined Identifier (ex: "a_token"):

In [ ]:
var client = clientBuilder.Build();
await client.Connect();
var urlStoreGrain = client.GetGrain<IUrlStoreGrain>("a_token");

The `urlStoreGrain` is the client stub object, call its `Task SetUrl(string shortenedRouteSegment, string fullUrl)` RPC method:

In [ ]:
await urlStoreGrain.SetUrl("a_token", @"https://www.google.com");

Call another RPC method `Task<string> GetUrl();` to get just stored long URL address:

In [ ]:
var longUrl = await urlStoreGrain.GetUrl();
display("return result url is: " + longUrl)

return result url is: https://www.google.com

Close connection of RPC client stub:

In [ ]:
await client.Close();

Shutdown server:

In [ ]:
await host.StopAsync();